In [ ]:
!pip install gpytorch
!pip install ConfigSpace
!pip install openml

In [ ]:
!pip install tabpfn

In [ ]:
!pip install scikit-learn==1.1.3


In [ ]:
!pip install ucimlrepo

In [ ]:
import sys
import numpy as np
from pathlib import Path
import pandas as pd
import torch

import os
import time
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from pathlib import Path

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import train_test_split

import tabpfn
from tabpfn import TabPFNClassifier


In [ ]:
from ucimlrepo import fetch_ucirepo 

adult = fetch_ucirepo(id=2) 

X = adult.data.features
y = adult.data.targets 
  

df = pd.concat([X,y],axis=1)

df = df.dropna()

df['income'] = df['income'].str.replace(r'K\.', 'K', regex=True)


In [ ]:
#데이터 세트 1000개 이상인 경우 실행 안되므로 Sampling 쪼개기

def stratified_sample(df, stratify_col, n_per_class, random_state=None, replace=False):
    return df.groupby(stratify_col).sample(n=n_per_class, replace=replace, random_state=random_state)


df = stratified_sample(df, stratify_col='income', n_per_class=400, random_state=123)


df = df.dropna()

In [ ]:
X = df[['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       ]]
y = df[['income']]

In [ ]:
#label을 0,1로 변환
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()

y_label = le.fit_transform(y.astype(str)) 



In [ ]:
#categorical data를 숫자로 변환후 변환된 값으로 데이터세트 칼럼 구성
for i in ['workclass','education',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'native-country']:
    X[f'{i}_cat'] = le.fit_transform(X[f'{i}'].astype(str))
X = X[['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week',
      'workclass_cat', 'education_cat', 'marital-status_cat',
      'occupation_cat', 'relationship_cat', 'race_cat', 'sex_cat',
      'native-country_cat']]

In [ ]:
max_configurations = X_train.shape[1] * len(set(y_train))
print("최대 N_ensemble_configurations:", max_configurations)

In [ ]:
# N_ensemble_configurations defines how many estimators are averaged, it is bounded by #features * #classes
# more ensemble members are slower, but more accurate
classifier = TabPFNClassifier(device='cuda', N_ensemble_configurations=32)

In [ ]:
start = time.time()
classifier.fit(X_train, y_train)
y_eval, p_eval = classifier.predict(X_test, return_winning_probability=True)
print('Prediction time: ', time.time() - start, 'Accuracy', accuracy_score(y_test, y_eval), 'AUROC', roc_auc_score(y_test,y_eval))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_cat, y_label, test_size=0.2, random_state=42)

In [ ]:
max_configurations = X_train.shape[1] * len(set(y_train))
print("최대 N_ensemble_configurations:", max_configurations)

In [ ]:
# N_ensemble_configurations defines how many estimators are averaged, it is bounded by #features * #classes
# more ensemble members are slower, but more accurate
classifier = TabPFNClassifier(device='cuda', N_ensemble_configurations=32)

In [ ]:
start = time.time()
classifier.fit(X_train, y_train)
y_eval, p_eval = classifier.predict(X_test, return_winning_probability=True)
print('Prediction time: ', time.time() - start, 'Accuracy', accuracy_score(y_test, y_eval))


'''
    - 기계학습특론 : 공정성 지표 metric example 코드
    - 공정성 지표 :
       1) Equal Opportunity (EOP)
       2) Equalized Odds (EO)
       3) Demographic Parity (DP)

    - Input :
       - prediction : 예측값
       - true_label : 실측값
       - sensitive_feature (= protected_attribute_train / test )
       - target_label : label 선택한 값 ( 1 , 0 )

    - Output :
       - 3 가지 지표 : 퍼센트값 (%)
        1) Equal Opportunity (EOP)
        2) Equalized Odds (EO)
        3) Demographic Parity (DP)
'''


In [ ]:

'''
    평가함수
'''


def equalized_odds_difference(predictions, true_labels, sensitive_features, target_label):

    binary_predictions = np.where(predictions > 0.5, 1, 0)
    positive_mask = true_labels == target_label
    tpr_0 = np.mean(binary_predictions[(sensitive_features == 0) & positive_mask] == target_label)
    tpr_1 = np.mean(binary_predictions[(sensitive_features == 1) & positive_mask] == target_label)

    return np.abs(tpr_0 - tpr_1)

def demographic_parity_difference(predictions, sensitive_features):

    binary_predictions = np.where(predictions > 0.5, 1, 0)

    rate_0 = np.mean(binary_predictions[sensitive_features == 0])
    rate_1 = np.mean(binary_predictions[sensitive_features == 1])

    return np.abs(rate_0 - rate_1)



"""지표 산출에 필요한 input 인자 생성 -> 'sex_cat' """


sensitive_attribute = 'sex_cat'
protected_attribute_train = X_train[sensitive_attribute]
protected_attribute_test = X_test[sensitive_attribute]




delta_equal_opportunity = equalized_odds_difference(y_eval, y_test, protected_attribute_test,
                                                    target_label=1)

delta_equalized_odds_negative = equalized_odds_difference(y_eval, y_test,
                                                          protected_attribute_test, target_label=0)

delta_demographic_parity = demographic_parity_difference(y_eval, protected_attribute_test)

delta_equalized_odds = max(delta_equal_opportunity, delta_equalized_odds_negative)




log_message = (f'delta_equalized_odds {delta_equalized_odds:.2%}\t delta_equal_opportunity {delta_equal_opportunity:.2%}'f'\t delta_demographic_parity {delta_demographic_parity:.2%}\t')



print(log_message)

성능 개선 방법
- N_ensemble_configurations 바꾸기 (max : 특성개수 * labeling 종류 개수)
- train/test 비율 변화 (train이 많을수록 좋아지긴 함)
- class 불균형 맞추기
- 여러번 샘플링 해서 결과를 투표로 결정
